# Exploratory analysis of the data sets related to FP7 projects for the Mobility Report 3

Author: Damir Valput
Date: 28 May 2020

In [1]:
#!pip3 install pandas_profiling

In [3]:
import re
import numpy as np
import pandas as pd
import pandas_profiling

import pickle

import matplotlib.pyplot as plt

import os
import glob

#NLTK
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

In [5]:
# all CSVs in the repo

path = "datasets/data_may20/fp7_raw/"
csvs = [f for f in glob.glob(path + "*.csv", recursive = False)]
print(len(csvs))

excels = [f for f in glob.glob(path + "*.xlsx", recursive = False)]
print(len(excels))

3
3


# Load the full set of FP7 projects (not filtered for mobility)

In [7]:
"""
The new database is stored in the folder 
    - "datasets/data_oct19/h2020"
    - "datasets/data_oct19/fp7"
    - "datasets/data_oct19/reference_data"

"""

# the project file (basic info) - CSV is poorly formatted, so reading xlsx!!

loc = "datasets/data_may20/fp7_raw/cordis-fp7projects.csv"
df_fp7 = pd.read_csv(loc, sep=";")
"""
df_fp7 = pd.read_excel(loc, header=0, names=['rcn', 'id', 'acronym', 'status', 'programme', 'topics', \
                                             'frameworkProgramme', 'title', 'startDate', 'endDate', 'projectUrl', \
                                            'objective', 'totalCost', 'ecMaxContribution', 'call', 'fundingScheme', \
                                            'coordinator', 'coordinatorCountry', 'participants', \
                                             'participantCountries', 'subjects'])
"""

df_fp7.head()

,rcn,id,acronym,status,programme,topics,frameworkProgramme,title,startDate,endDate,...,objective,totalCost,ecMaxContribution,call,fundingScheme,coordinator,coordinatorCountry,participants,participantCountries,subjects
0,203726,115760,ZAPI,ONG,FP7-JTI,IMI-JU-11-2013-04,FP7,Zoonotic Anticipation and Preparedness Initiative,2015-03-01,2020-02-29,...,\nEmerging infectious diseases are occurring a...,22360163,9538688,IMI-JU-11-2013,JTI-CP-IMI,MERIAL SAS,FR,ACADEMISCH ZIEKENHUIS LEIDEN;STIFTUNG TIERAERZ...,NL;DE;SE;FR;ES,NaN
1,109748,320377,NetSat,ONG,FP7-IDEAS-ERC,ERC-AG-PE8,FP7,Networked Pico-Satellite Distributed System Co...,2014-08-01,2019-07-31,...,A paradigm shift is emerging in spacecraft eng...,2500000,2500000,ERC-2012-ADG_20120216,ERC-AG,Zentrum fuer Telematik e.V.,DE,JULIUS-MAXIMILIANS-UNIVERSITAT WURZBURG,DE,NaN
2,188675,615785,EMERGING SUBJECTS,ONG,FP7-IDEAS-ERC,ERC-CG-2013-SH2,FP7,Emerging Subjects of the New Economy: Tracing ...,2014-09-01,2019-06-30,...,This project examines how predicted economic g...,1658373,1658373,ERC-2013-CoG,ERC-CG,UNIVERSITY COLLEGE LONDON,UK,NaN,NaN,NaN
3,188672,615640,FOREFRONT,ONG,FP7-IDEAS-ERC,ERC-CG-2013-PE6,FP7,Frontiers of Extended Formulations,2014-09-01,2019-08-31,...,'Linear programming has proved to be an invalu...,1455479,1455479,ERC-2013-CoG,ERC-CG,UNIVERSITE LIBRE DE BRUXELLES,BE,NaN,NaN,NaN
4,189842,617196,CORRELMAT,ONG,FP7-IDEAS-ERC,ERC-CG-2013-PE3,FP7,Predictive electronic structure calculations f...,2014-07-01,2019-06-30,...,'Materials with strong electronic Coulomb corr...,1713600,1713600,ERC-2013-CoG,ERC-CG,ECOLE POLYTECHNIQUE,FR,NaN,NaN,NaN


In [8]:
print("Projects: ", str(df_fp7.shape[0]))
print("Columns per project: ", str(df_fp7.shape[1]))
print(list(df_fp7.columns))

Projects:  25778
Columns per project:  21
['rcn', 'id', 'acronym', 'status', 'programme', 'topics', 'frameworkProgramme', 'title', 'startDate', 'endDate', 'projectUrl', 'objective', 'totalCost', 'ecMaxContribution', 'call', 'fundingScheme', 'coordinator', 'coordinatorCountry', 'participants', 'participantCountries', 'subjects']


In [9]:
df_fp7.describe()

,rcn,id,subjects
count,25778.000000,25778.000000,0.0
mean,108674.010823,328680.648770,NaN
std,28349.164969,138540.352039,NaN
min,85231.000000,100016.000000,NaN
25%,94138.250000,244593.250000,NaN
50%,100905.500000,281303.500000,NaN
75%,107693.750000,323413.750000,NaN
max,212013.000000,921672.000000,NaN


In [13]:
profile = df_fp7.profile_report(title='FP7 projects: summary statistics')

In [14]:
dirName = "pandas_reports"

try:
    os.makedirs(dirName)    
    print("Directory " , dirName ,  " Created ")
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  pandas_reports  Created 


In [15]:
profile.to_file(output_file="pandas_reports/fp7_projects_summary_report.html")

# Investigate duplicates and missing values

In [46]:
title_duplicates = df_fp7[df_fp7.duplicated(subset=['title'], keep=False)]

title_duplicates.sort_values("title").shape

(309, 21)

In [47]:
title_obj_duplicates = df_fp7[df_fp7.duplicated(subset=['title', 'objective'], keep=False)]

title_obj_duplicates.sort_values("title").shape

(120, 21)

In [18]:
# missing value in objectives

df_fp7[df_fp7['objective'].isna()]

,rcn,id,acronym,status,programme,topics,frameworkProgramme,title,startDate,endDate,...,objective,totalCost,ecMaxContribution,call,fundingScheme,coordinator,coordinatorCountry,participants,participantCountries,subjects
24401,96280,260887,SEENANOLIFEINACTION,ONG,FP7-IDEAS-ERC,ERC-SG-LS1,FP7,Real-Time Studies of Biological NanoMachines i...,2011-01-01,2015-12-31,...,NaN,1398392,1398392,ERC-2010-StG_20091118,ERC-SG,CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE CNRS,FR,NaN,NaN,NaN
25332,107494,100204,PS.HI.E.L.D.,ONG,FP7-JTI,SP1-JTI-ARTEMIS-2009-6,FP7,pilot embedded Systems arcHItecturE for multi-...,2010-06-01,2011-11-30,...,NaN,"5392810,07","900599,27",ARTEMIS-2009-1,JTI-CP-ARTEMIS,MOVATION AS,NO,TECNOLOGIE NELLE RETI E NEI SISTEMI T.R.S. SPA...,IT;EL;ES;SI;PT;NO,NaN


In [19]:
# remove missing values
fp7_clean = df_fp7[df_fp7['objective'].notnull()]
fp7_clean.shape

(25776, 21)

In [20]:
# projects with duplicate title and objective
duplicate_to_remove = df_fp7[df_fp7.duplicated(subset=['title','objective'], keep='first')]['id']
print(len(duplicate_to_remove))
fp7_clean = fp7_clean[~fp7_clean['id'].isin(duplicate_to_remove)]
fp7_clean.shape

60


(25716, 21)

In [21]:
# save cleaned projects file
#fp7_clean.to_csv("datasets/data_oct19/fp7_clean/projects_clean.csv")

In [22]:
# LIST OF project IDs that are kept for further analysis

fp7_project_ids = set(fp7_clean.id)
len(fp7_project_ids)

25716

## Load summary reports: FP7

In [24]:
loc = "datasets/data_may20/fp7_raw/cordis-fp7reports.csv"
reports = pd.read_csv(loc, sep=";")


reports.head()

,rcn,language,title,teaser,summary,workPerformed,finalResults,lastUpdateDate,country,projectID,projectAcronym,programme,topics,relatedFile,url
0,401249,en,Final Report Summary - DELPOWIO (Delegation of...,How powerful are international organizations (...,How powerful are international organizations (...,NaN,NaN,2019-08-13 15:31:10,DE,312368.0,DELPOWIO,FP7-IDEAS-ERC,ERC-SG-SH2,NaN,NaN
1,401242,en,Final Report Summary - IONPAIRSATCATALYSIS (De...,Ion pairs are nearly omnipresent in chemistry ...,Ion pairs are nearly omnipresent in chemistry ...,NaN,NaN,2019-08-13 15:31:08,DE,614182.0,IONPAIRSATCATALYSIS,FP7-IDEAS-ERC,ERC-CG-2013-PE5,NaN,NaN
2,401243,en,Final Report Summary - BACTERIAL RESPONSE (New...,Bacteria in nature exhibit remarkable capacity...,Bacteria in nature exhibit remarkable capacity...,NaN,NaN,2019-08-13 15:31:08,IL,339984.0,BACTERIAL RESPONSE,FP7-IDEAS-ERC,ERC-AG-LS6,NaN,NaN
3,401226,en,Final Report Summary - PACART (Free space phot...,The project PACART was devoted to enhancing th...,The project PACART was devoted to enhancing th...,NaN,NaN,2019-08-13 15:30:49,DE,340625.0,PACART,FP7-IDEAS-ERC,ERC-AG-PE2,NaN,NaN
4,401235,en,Final Report Summary - EQUALIS (EQualIS : Enha...,"The ubiquitous use of computerized systems, an...","The ubiquitous use of computerized systems, an...",NaN,NaN,2019-08-13 15:30:49,FR,308087.0,EQUALIS,FP7-IDEAS-ERC,ERC-SG-PE6,NaN,NaN


In [25]:
#reports.tail()

In [26]:
reports.describe()

,workPerformed,projectID
count,1.0,21874.000000
mean,605134.0,331791.444272
std,NaN,137414.068028
min,605134.0,200130.000000
25%,605134.0,247471.250000
50%,605134.0,284233.500000
75%,605134.0,326107.750000
max,605134.0,921672.000000


In [27]:
#profile = reports.profile_report(title='FP7 projects: summary statistics')
#profile.to_file(output_file="pandas_reports/fp7_projects_summary_report.html")

In [28]:
# profile report provided by pandas_profiling
reports.profile_report()

In [29]:
# this dataset has 8 duplicate rows
#reports[reports.duplicated(keep=False)]

# remove all of those rows as they are all NaN values
reports.dropna(axis=0, thresh=5, inplace=True) # keep if at least thresh columns are not-nan
reports.shape

(21876, 15)

In [30]:
# drop any with projectID NaN
reports.dropna(axis=0, subset=["projectID"], inplace=True)
reports.shape

(21874, 15)

In [31]:
reports[~reports.relatedFile.isna()].head()

,rcn,language,title,teaser,summary,workPerformed,finalResults,lastUpdateDate,country,projectID,projectAcronym,programme,topics,relatedFile,url
23,401208,en,Final Report Summary - PROFNAIT (Development o...,PROFNAIT is a European Union-funded consortium...,Executive Summary:\nPROFNAIT is a European Uni...,NaN,NaN,2019-08-12 11:44:39,NO,305986.0,PROFNAIT,FP7-HEALTH,HEALTH.2012.2.4.4-1,/docs/results/305/305986/final1-profnait-final...,NaN
46,351553,en,Final Report Summary - PACE (Comparing the eff...,"PACE ""Palliative Care for Older people"" is a E...","Executive Summary:\nPACE ""Palliative Care for ...",NaN,NaN,2019-08-08 15:09:04,BE,603111.0,PACE,FP7-HEALTH,HEALTH.2013.3.1-1,/docs/results/603/603111/final1-1-pace-steps-t...,NaN
59,401227,en,Final Report Summary - FELLOWSEA (Programme fo...,The Programme for Post-Doctoral Talent Attract...,The Programme for Post-Doctoral Talent Attract...,NaN,NaN,2019-08-13 15:30:58,ES,600391.0,FELLOWSEA,FP7-PEOPLE,FP7-PEOPLE-2012-COFUND,/docs/results/600/600391/final1-fellowsea-logo...,NaN
73,401204,en,Final Report Summary - GENOVATE (Transforming ...,GENOVATE is an action-research project involvi...,Executive Summary:\nGENOVATE is an action-rese...,NaN,NaN,2019-08-12 11:45:12,UK,321378.0,GENOVATE,FP7-SIS,SiS.2012.2.1.1-1,/docs/results/321/321378/final1-final-genovate...,NaN
77,401202,en,Final Report Summary - EUROFANCOLEN (Phase I/I...,Fanconi anemia (FA) is a rare inherited syndro...,Executive Summary:\nFanconi anemia (FA) is a r...,NaN,NaN,2019-08-12 11:44:50,ES,305421.0,EUROFANCOLEN,FP7-HEALTH,HEALTH.2012.2.4.4-1,/docs/results/305/305421/final1-figures-final-...,NaN


In [32]:
# keep projects only in fp7_project_ids
reports_clean = reports[reports.projectID.isin(fp7_project_ids)]

reports_clean.shape

(21825, 15)

In [33]:
# drop fields from the clean file: ['language', 'teaser', 'country', 'programme', 'topics', etc.]

reports_clean.drop(['language', 'teaser', 'country', 'programme', 'topics', 'workPerformed', 'finalResults', 'url'], axis=1, inplace=True)
#reports_clean.drop(['language', 'teaser', 'country', 'programme', 'topics'], axis=1, inplace=True)
reports_clean.shape

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(21825, 7)

In [34]:
# There are rows with duplicated ProjectID!!
id_duplicates = reports_clean[reports_clean.duplicated(subset=['projectID'], keep=False)]

id_duplicates.sort_values("projectID")

,rcn,title,summary,lastUpdateDate,projectID,projectAcronym,relatedFile
21816,51439,Periodic Report Summary 2 - KV CHANNELS & MEMO...,The aim of this research proposal was to deter...,2017-02-16 16:05:24,200632.0,KV CHANNELS & MEMORY,NaN
21817,45774,Periodic Report Summary 1 - KV CHANNELS & MEMO...,A summary description of the project objective...,2018-07-06 13:52:51,200632.0,KV CHANNELS & MEMORY,NaN
17285,51441,Periodic Report Summary 2 - CD49D IN DIABETES ...,Type I (autoimmune) diabetes remains an import...,2017-02-16 16:05:25,200876.0,CD49D IN DIABETES,NaN
19483,46421,Periodic Report Summary 1 - CD49D IN DIABETES ...,A grant of EUR 100.000 was approved to explore...,2017-02-16 13:14:53,200876.0,CD49D IN DIABETES,NaN
21636,52821,Periodic Report Summary - STOPLATENT-TB (Laten...,It has been estimated that one third of the hu...,2017-02-16 16:47:26,200999.0,STOPLATENT-TB,NaN
...,...,...,...,...,...,...,...
570,197084,Final Report Summary - CERTIFHY (Developing a ...,Executive Summary:\nThe development of hydroge...,2019-04-25 12:05:09,633107.0,CertifHy,/docs/results/633/633107/final1-publishablesum...
9210,183712,Periodic Report Summary 1 - CERTIFHY (Developi...,Project Context and Objectives:\nThe developme...,2017-02-21 16:33:20,633107.0,CertifHy,NaN
4387,197913,Periodic Report Summary 3 - REFLECTOCFP16 (In-...,Project Context and Objectives:\nTo measure th...,2017-05-16 11:52:44,641067.0,REFLECTOCFP16,NaN
20744,187813,Periodic Report Summary 2 - REFLECTOCFP16 (In-...,Project Context and Objectives:\nLaminar Flow ...,2017-02-21 17:08:21,641067.0,REFLECTOCFP16,NaN


In [35]:
# Group all the projects of the same ID in one row!
aggregation_functions = {'summary': 'sum', 'lastUpdateDate': 'last', 'rcn': 'first',\
                         'title':'sum', 'projectAcronym':'first'}
reports_clean_final = reports_clean.groupby(reports_clean['projectID']).aggregate(aggregation_functions)

In [36]:
reports_clean_final.shape

(21179, 5)

In [37]:
reports_clean_final.head()

,summary,lastUpdateDate,rcn,title,projectAcronym
projectID,,,,,
200130.0,The Secret Society of Animal Cells:\nMost of u...,2017-02-21 15:28:33,180732,Final Report Summary - SUPERCOMPETITORS (GENET...,SUPERCOMPETITORS
200141.0,The QuESpace project aims to quantify energy c...,2017-02-21 15:28:32,180731,Final Report Summary - QUESPACE (Quantifying E...,QUESPACE
200165.0,The accident at Chernobyl nuclear power plant ...,2017-02-16 16:03:10,50820,Periodic Report Summary - PROTEOMICS OF CHERNO...,PROTEOMICS OF CHERNO
200176.0,Project context and objectives\n\nThis project...,2018-06-29 18:16:13,59799,Final Report Summary - LOMAIVITICINS (The synt...,LOMAIVITICINS
200179.0,Transformation by ras oncogenes induces the de...,2018-06-06 18:18:42,230834,Final Report Summary - RAS:EFFECTORS (RAS supe...,RAS:EFFECTORS


In [38]:
#reports_clean_final.to_csv("datasets/data_oct19/fp7_clean/reports_clean.csv")

# Merge basic project info and reports

In [39]:
fp7_clean.shape

(25716, 21)

In [40]:
fp7_full = fp7_clean.merge(reports_clean_final, how = 'left', left_on="id", right_on = "projectID", suffixes=("", "_report"))
fp7_full.shape

(25716, 26)

In [41]:
fp7_full.head()

,rcn,id,acronym,status,programme,topics,frameworkProgramme,title,startDate,endDate,...,coordinator,coordinatorCountry,participants,participantCountries,subjects,summary,lastUpdateDate,rcn_report,title_report,projectAcronym
0,203726,115760,ZAPI,ONG,FP7-JTI,IMI-JU-11-2013-04,FP7,Zoonotic Anticipation and Preparedness Initiative,2015-03-01,2020-02-29,...,MERIAL SAS,FR,ACADEMISCH ZIEKENHUIS LEIDEN;STIFTUNG TIERAERZ...,NL;DE;SE;FR;ES,NaN,NaN,NaN,NaN,NaN,NaN
1,109748,320377,NetSat,ONG,FP7-IDEAS-ERC,ERC-AG-PE8,FP7,Networked Pico-Satellite Distributed System Co...,2014-08-01,2019-07-31,...,Zentrum fuer Telematik e.V.,DE,JULIUS-MAXIMILIANS-UNIVERSITAT WURZBURG,DE,NaN,Multi-satellite systems are currently gaining ...,2017-10-30 17:19:45,202963,Periodic Report Summary 2 - NETSAT (Networked ...,NetSat
2,188675,615785,EMERGING SUBJECTS,ONG,FP7-IDEAS-ERC,ERC-CG-2013-SH2,FP7,Emerging Subjects of the New Economy: Tracing ...,2014-09-01,2019-06-30,...,UNIVERSITY COLLEGE LONDON,UK,NaN,NaN,NaN,Emerging Subjects has reached a mid-point in i...,2018-01-15 17:25:32,213801,Periodic Report Summary 2 - EMERGING SUBJECTS ...,EMERGING SUBJECTS
3,188672,615640,FOREFRONT,ONG,FP7-IDEAS-ERC,ERC-CG-2013-PE6,FP7,Frontiers of Extended Formulations,2014-09-01,2019-08-31,...,UNIVERSITE LIBRE DE BRUXELLES,BE,NaN,NaN,NaN,The FOREFRONT project (Frontiers of Extended F...,2017-07-24 18:20:56,201439,Periodic Report Summary 2 - FOREFRONT (Frontie...,FOREFRONT
4,189842,617196,CORRELMAT,ONG,FP7-IDEAS-ERC,ERC-CG-2013-PE3,FP7,Predictive electronic structure calculations f...,2014-07-01,2019-06-30,...,ECOLE POLYTECHNIQUE,FR,NaN,NaN,NaN,"“Correlated electron materials”, i.e. compound...",2017-05-16 10:26:14,197668,Mid-Term Report Summary - CORRELMAT (Predictiv...,CORRELMAT


In [43]:
dirName = "datasets/data_may20/fp7_clean"

try:
    os.makedirs(dirName)    
    print("Directory " , dirName ,  " Created ")
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  datasets/data_may20/fp7_clean  Created 


In [44]:
fp7_full.to_csv("datasets/data_may20/fp7_clean/fp7_full.csv", index=False)

# Read other data - dictionary placement

df_fp7 - basic h2020 projects data - in a separate df
reports - report summaries of FP7 - in a separate df


KEYS --> all dataframes placed in the dictionary fp7:
- organisations
- reports - report summaries data, as in reports
- projects - basic project data, as in df_fp7

In [45]:
keys = ['organisations', 'reports', 'projects']

fp7 = dict()
for key, loc in zip(keys, excels):
    fp7[key] = pd.read_excel(loc)
    
len(fp7)

3

## Completeness of data

In [49]:
fp7_full.columns.values

array(['rcn', 'id', 'acronym', 'status', 'programme', 'topics',
       'frameworkProgramme', 'title', 'startDate', 'endDate',
       'projectUrl', 'objective', 'totalCost', 'ecMaxContribution',
       'call', 'fundingScheme', 'coordinator', 'coordinatorCountry',
       'participants', 'participantCountries', 'subjects', 'summary',
       'lastUpdateDate', 'rcn_report', 'title_report', 'projectAcronym'],
      dtype=object)

In [50]:
fp7_full.projectUrl

0                                                      NaN
1        https://www.telematik-zentrum.de/index.php?id=...
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
25711                                                  NaN
25712                                                  NaN
25713                                                  NaN
25714                                                  NaN
25715                                                  NaN
Name: projectUrl, Length: 25716, dtype: object

In [54]:
# how many url's are missing?

sum(fp7_full.projectUrl.isna())

21277

In [55]:
fp7_full.shape

(25716, 26)